In [122]:
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sns
import datetime as dt


In [20]:
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_excel(r"C:\Users\MONSTER\Desktop\dataset\online_retail.xlsx")

In [23]:
df = df.copy()

In [24]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [30]:
df.shape

(541909, 8)

In [32]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [34]:
df["Description"].nunique()

4223

In [36]:
df["Description"].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER     2369
REGENCY CAKESTAND 3 TIER               2200
JUMBO BAG RED RETROSPOT                2159
PARTY BUNTING                          1727
LUNCH BAG RED RETROSPOT                1638
                                       ... 
Missing                                   1
historic computer difference?....se       1
DUSTY PINK CHRISTMAS TREE 30CM            1
WRAP BLUE RUSSIAN FOLKART                 1
PINK BERTIE MOBILE PHONE CHARM            1
Name: count, Length: 4223, dtype: int64

In [38]:
df.groupby("Description").agg({"Quantity" : "sum"})

,Quantity
Description,
20713,-400
4 PURPLE FLOCK DINNER CANDLES,144
50'S CHRISTMAS GIFT BAG LARGE,1913
DOLLY GIRL BEAKER,2448
I LOVE LONDON MINI BACKPACK,389
...,...
wrongly marked carton 22804,-256
wrongly marked. 23343 in box,-3100
wrongly sold (22719) barcode,170


In [40]:
df.groupby("Description").agg({"Quantity" : "sum"}).sort_values("Quantity", ascending = False)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039
...,...
Damaged,-7540
Printing smudges/thrown away,-9058
check,-12030


In [44]:
df["InvoiceNo"].nunique()

25900

In [52]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
df.groupby("InvoiceNo").agg({"TotalPrice" : "sum"})

,TotalPrice
InvoiceNo,
536365,139.120
536366,22.200
536367,278.730
536368,70.050
536369,17.850
...,...
C581484,-168469.600
C581490,-32.530
C581499,-224.690


In [56]:
###veri hazırlama 
df.dropna(inplace = True)

In [58]:
df = df[~df["InvoiceNo"].str.contains("C", na=False)]


In [62]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [68]:
today_date = dt.datetime(2010, 12, 11)

In [70]:
type(today_date)

datetime.datetime

In [78]:
####rfm metriklerinin hazırlanması 
rfm = df.groupby("CustomerID").agg({"InvoiceDate": lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                    "InvoiceNo" : lambda Invoice: Invoice.nunique(),
                                    "TotalPrice" : lambda TotalPrice: TotalPrice.sum()})

In [80]:
rfm.head()

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.000,-39,1,77183.600
12347.000,-362,7,4310.000
12348.000,-289,4,1797.240
12349.000,-346,1,1757.550
12350.000,-54,1,334.400


In [82]:
rfm.columns = "Recency", "Frequency", "Monetary"
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4339.000,-271.959,100.008,-364.000,-347.000,-314.000,-222.500,9.000
Frequency,4339.000,4.272,7.705,1.000,1.000,2.000,5.000,210.000
Monetary,4339.000,2053.793,8988.248,0.000,307.245,674.450,1661.640,280206.020


In [86]:
rfm = rfm[rfm["Monetary"] > 0 ]

In [88]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.000,-271.941,100.012,-364.000,-347.000,-314.000,-222.250,9.000
Frequency,4338.000,4.273,7.706,1.000,1.000,2.000,5.000,210.000
Monetary,4338.000,2054.266,8989.230,3.750,307.415,674.485,1661.740,280206.020


In [90]:
rfm.shape

(4338, 3)

In [94]:
###rfm metriklerinin hesaplanması 
rfm["Recency_Score"] = pd.qcut(rfm["Recency"], 5, labels= [5, 4, 3, 2, 1])
rfm.head()

,Recency,Frequency,Monetary,Recency_Score
CustomerID,,,,
12346.000,-39,1,77183.600,1
12347.000,-362,7,4310.000,5
12348.000,-289,4,1797.240,2
12349.000,-346,1,1757.550,4
12350.000,-54,1,334.400,1


In [96]:
rfm["Monetary_Score"] = pd.qcut(rfm["Monetary"], 5, labels= [1, 2 , 3, 4, 5])
rfm.head()

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score
CustomerID,,,,,
12346.000,-39,1,77183.600,1,5
12347.000,-362,7,4310.000,5,5
12348.000,-289,4,1797.240,2,4
12349.000,-346,1,1757.550,4,4
12350.000,-54,1,334.400,1,2


In [98]:
rfm["Frequency_Score"] = pd.qcut(rfm["Frequency"].rank(method = "first"), 5, labels= [1, 2 , 3, 4, 5])
rfm.head()

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score
CustomerID,,,,,,
12346.000,-39,1,77183.600,1,5,1
12347.000,-362,7,4310.000,5,5,5
12348.000,-289,4,1797.240,2,4,4
12349.000,-346,1,1757.550,4,4,1
12350.000,-54,1,334.400,1,2,1


In [102]:
rfm["RFM_SCORE"] = (rfm["Recency_Score"].astype(str) + rfm["Frequency_Score"].astype(str))

In [104]:
rfm[rfm["RFM_SCORE"] == "55"]

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE
CustomerID,,,,,,,
12347.000,-362,7,4310.000,5,5,5,55
12362.000,-361,10,5226.230,5,5,5,55
12417.000,-361,9,3649.100,5,5,5,55
12423.000,-364,8,1859.310,5,4,5,55
12433.000,-364,7,13375.870,5,5,5,55
...,...,...,...,...,...,...,...
18230.000,-355,7,2810.200,5,5,5,55
18241.000,-355,17,2073.090,5,5,5,55
18245.000,-357,7,2567.060,5,5,5,55


In [106]:
rfm[rfm["RFM_SCORE"] == "11"]

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE
CustomerID,,,,,,,
12346.000,-39,1,77183.600,1,5,1,11
12350.000,-54,1,334.400,1,2,1,11
12353.000,-160,1,89.000,1,1,1,11
12354.000,-132,1,1079.400,1,4,1,11
12355.000,-150,1,459.400,1,2,1,11
...,...,...,...,...,...,...,...
15717.000,-130,1,150.300,1,1,1,11
15723.000,0,1,203.100,1,1,1,11
15729.000,-152,1,85.000,1,1,1,11


In [108]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.000,-271.941,100.012,-364.000,-347.000,-314.000,-222.250,9.000
Frequency,4338.000,4.273,7.706,1.000,1.000,2.000,5.000,210.000
Monetary,4338.000,2054.266,8989.230,3.750,307.415,674.485,1661.740,280206.020


In [110]:
####segmentlerin oluşturulması 
# Örnek segment eşleştirme sözlüğü
seg_map = {
    r'[1-2][1-2]': 'hibernating',         # düşük recency, düşük frequency
    r'[1-2][3-4]': 'at risk',             # düşük recency, orta frequency
    r'[1-2]5': 'can\'t lose',             # düşük recency, yüksek frequency
    r'3[1-2]': 'about to sleep',          # orta recency, düşük frequency
    r'33': 'need attention',             # orta recency, orta frequency
    r'[3-4][4-5]': 'loyal customers',     # orta recency, yüksek frequency
    r'41': 'promising',                  # iyi recency, düşük frequency
    r'51': 'new customers',              # en iyi recency, düşük frequency
    r'[4-5][2-3]': 'potential loyalists', # iyi recency, orta frequency
    r'5[4-5]': 'champions'               # en iyi recency ve frequency
}

# Segmentleri uygulamak için regex eşlemesi yapılır
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)


In [112]:
rfm[["segment", "Recency", "Monetary", "Frequency"]].groupby("segment").agg(["mean", "count"])

Recency       Monetary       Frequency      
                        mean count     mean count      mean count
segment                                                          
about to sleep      -311.688   352  471.994   352     1.162   352
at risk             -211.214   593 1084.535   593     2.879   593
can't lose          -232.032    63 2796.156    63     8.381    63
champions           -358.638   633 6857.935   633    12.417   633
hibernating         -147.395  1071  488.643  1071     1.102  1071
loyal customers     -331.392   819 2864.248   819     6.480   819
need attention      -312.572   187  897.628   187     2.326   187
new customers       -357.571    42  388.213    42     1.000    42
potential loyalists -347.601   484 1041.222   484     2.010   484
promising           -341.489    94  294.008    94     1.000    94

In [114]:
rfm[rfm["segment"] == "need attention"].head()

,Recency,Frequency,Monetary,Recency_Score,Monetary_Score,Frequency_Score,RFM_SCORE,segment
CustomerID,,,,,,,,
12360.000,-312,3,2662.060,3,5,3,33,need attention
12372.000,-293,3,1298.040,3,4,3,33,need attention
12413.000,-298,3,758.100,3,3,3,33,need attention
12456.000,-320,3,3181.040,3,5,3,33,need attention
12536.000,-321,3,12601.830,3,5,3,33,need attention


In [116]:
rfm[rfm["segment"] == "new customers"].index

Index([12367.0, 12442.0, 12478.0, 12479.0, 12558.0, 12587.0, 12650.0, 12660.0,
       12713.0, 12953.0, 12966.0, 13017.0, 13153.0, 13188.0, 13298.0, 13349.0,
       13428.0, 13436.0, 13560.0, 13790.0, 13844.0, 14087.0, 14093.0, 14204.0,
       14219.0, 14349.0, 14480.0, 14569.0, 14578.0, 14601.0, 14804.0, 14954.0,
       15097.0, 15148.0, 15195.0, 15318.0, 15471.0, 15520.0, 15539.0, 15540.0,
       15619.0, 15721.0],
      dtype='float64', name='CustomerID')

In [126]:
new_df = pd.DataFrame()


In [132]:
new_df["new_customer_id"] = rfm[rfm["segment"] == "new customers"].index


In [134]:
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

In [136]:
new_df.to_csv("new_customers.csv")

In [138]:
rfm.to_csv("rfm.csv")